In [1]:
!pip install mlcrate

In [2]:
import pandas as pd
import numpy as np
#import gc

import mlcrate as mlc
import pickle as pkl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Embedding, Dense, Flatten, Concatenate,
                                     Dot, Reshape, Add, Subtract, BatchNormalization)
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model

embedding_reg = 0.0002
kernel_reg = 0.1
out_dim=2
weight_matrix = np.arange(12).reshape((-1,2))

def get_embed(x_input, x_size, out_dim,test_weights=False):
    # x_input is index of input (either user or item)
    # x_size is length of vocabulary (e.g. total number of users or items)
    # test_weights is a demo flag to show results with predefined weights
    # out_dim is size of embedding vectors
    if x_size > 0: #category
        if test_weights & (out_dim<=2):
            embed = Embedding(0, out_dim, input_length=1,
                          weights=[weight_matrix[:x_size,:out_dim]],
                          embeddings_regularizer=l2(embedding_reg))(x_input)
        else:
            embed = Embedding(x_size, out_dim, input_length=1,
                              embeddings_regularizer=l2(embedding_reg))(x_input)
        embed = Flatten()(embed)
    else:
        embed = Dense(out_dim, kernel_regularizer=l2(embedding_reg))(x_input)
    return embed
def build_model(f_size):
    k_latent = 3
    kernel_reg = 0.01

    input_x = [Input(shape=(1,)) for _ in range(len(f_size))]
    lin_terms = [get_embed(x, size, 1) for (x, size) in zip(input_x, f_size)]
    factors = [get_embed(x, size, k_latent) for (x, size) in zip(input_x, f_size)]

    # Sum of factors
    s = Add()(factors)
    diffs = [Subtract()([s, x]) for x in factors]
    dots = [Dot(axes=1)([d, x]) for d, x in zip(diffs, factors)]

    # Concatenate linear terms and dot products
    x = Concatenate()(lin_terms + dots)

    # BatchNormalization expects a fixed feature size
    x = Dense(4, activation='linear')(x)  # Aligning features to a fixed size
    x = BatchNormalization()(x)          # Batch normalization applied to a fixed-size tensor

    # Final output layer
    output = Dense(1, activation='relu', kernel_regularizer=l2(kernel_reg))(x)

    model = Model(inputs=input_x, outputs=[output])
    model.compile(optimizer=Adam(clipnorm=0.01, learning_rate=0.001),
                  loss='mean_squared_error')
    return model


train_df = pd.read_csv("train_sample.csv")
test_df = pd.read_csv("test_sample.csv")

X_train = train_df[['feature_0', 'feature_1', 'feature_2']].values
y_train = train_df['target'].values
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_test = test_df[['feature_0', 'feature_1', 'feature_2']].values

X_train_list = np.hsplit(X_train, 3)
X_test_list = np.hsplit(X_test, 3)
X_val_list = np.hsplit(X_val,3)
features = ['feature0', "feature1", "feature2"]
f_size = [int(np.max(X_train_list[i])) + 1 for i in range(3)]
print(X_train_list)
n_epochs = 100

batch_size = 77
print('Batch size: ',batch_size)
model = build_model(f_size)
earlystopper = EarlyStopping(patience=2, verbose=0)
model.fit(X_train_list, y_train, epochs=n_epochs, batch_size=batch_size, verbose=0,
          validation_data=(X_val_list, y_val))
best_epoch = earlystopper.stopped_epoch
print(best_epoch)
print('RMSE',model.evaluate(X_val_list, y_val,
                            batch_size=batch_size,verbose=0)**0.5)
prediction = model.predict(X_test_list)
sub = pd.read_csv('test_sample.csv',usecols=['ID'])
sub['target'] = prediction
sub.to_csv('submission.csv',index=False)

[array([[44],
       [23],
       [39],
       ...,
       [32],
       [18],
       [30]]), array([[24],
       [23],
       [ 7],
       ...,
       [17],
       [12],
       [31]]), array([[10],
       [ 6],
       [37],
       ...,
       [ 1],
       [ 9],
       [ 3]])]
Batch size:  77


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


0
RMSE 0.2726338292606943
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
